In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import imageio

# ----------------------------
# 1. Define a tiny MuJoCo XML
# ----------------------------
BALL_XML = """
<mujoco model="ball_drop">
  <option timestep="0.01" gravity="0 0 -9.81"/>
  <worldbody>
    <!-- Ground plane -->
    <geom name="floor" type="plane" size="2 2 0.1" rgba="0.7 0.9 0.7 1"/>
    
    <!-- Ball -->
    <body name="ball" pos="0 0 1">
      <freejoint/>
      <geom type="sphere" size="0.05" rgba="0.9 0.2 0.2 1" density="1000"/>
    </body>
  </worldbody>
</mujoco>
"""

In [2]:
# ----------------------------
# 2. Load model and data
# ----------------------------
model = mujoco.MjModel.from_xml_string(BALL_XML)
data = mujoco.MjData(model)

print("Number of joints:", model.njnt)
print("Initial ball position (z):", data.qpos[2])


Number of joints: 1
Initial ball position (z): 1.0


In [ ]:

# ----------------------------
# 3. Step the physics
# ----------------------------
frames = []
for step in range(150):
    mujoco.mj_step(model, data)   # advance simulation
    
    # Render RGB frame (headless)
    width, height = 320, 240
    rgb = np.zeros((height, width, 3), dtype=np.uint8)
    depth = np.zeros((height, width), dtype=np.float32)

    cam = mujoco.MjvCamera()
    opt = mujoco.MjvOption()
    scn = mujoco.MjvScene(model, 2000)
    con = mujoco.MjrContext(model, mujoco.mjtFontScale.mjFONTSCALE_150.value)

    cam.lookat[:] = [0, 0, 0.5]   # camera target
    cam.distance = 1.5
    cam.azimuth = 90
    cam.elevation = -20

    mujoco.mjv_updateScene(model, data, opt, None, cam,
                           mujoco.mjtCatBit.mjCAT_ALL.value, scn)
    mujoco.mjr_render(mujoco.MjrRect(0, 0, width, height), scn, con)
    mujoco.mjr_readPixels(rgb, depth, mujoco.MjrRect(0, 0, width, height), con)

    mujoco.mjr_freeContext(con)
    scn.free()

    frames.append(np.flipud(rgb))  # flip vertically

# ----------------------------
# 4. Show last frame inline
# ----------------------------
plt.imshow(frames[-])
plt.axis("off")
plt.show()

# ----------------------------
# 5. Save as GIF
# ----------------------------
imageio.mimsave("ball_drop.gif", frames, fps=30)
print("Saved GIF: ball_drop.gif")
